In [10]:
import xlwings as xw
import pandas as pd
import pyautocad

Импорт таблицы в Autocad

In [6]:
sheet = xw.sheets.active
df = sheet.range('A1').expand().options(pd.DataFrame, header=1, index=False).value
df['Номер'] = df['Номер'].apply(lambda x: str(int(float(x))))
df = df[['Номер', 'Наименование', 'Количество', 'Высота', 'Толщина', 'Состояние', 'Действие']]
df

,Номер,Наименование,Количество,Высота,Толщина,Состояние,Действие
0,316,береза,7.4 м.кв.,1.0,0.05,Хорошее,Удаление
1,317,ольха,7.4 м.кв.,2.5,0.08,Хорошее,Удаление
2,307,береза,30.7 м.кв.,1.5,0.05,Хорошее,Удаление
3,308,ольха,30.7 м.кв.,2.0,0.06,Хорошее,Удаление
4,305,Яблоня домашняя,2 ствола,1.5х2,0.04х2,Хорошее,Удаление
5,306,Яблоня домашняя,4,1.5х4,0.04х4,Хорошее,Удаление
6,309,Яблоня домашняя,3,1.5х3,0.03х3,Хорошее,Удаление
7,312,Яблоня домашняя,2 ствола,1.5х2,0.04х2,Хорошее; плохое,Удаление
8,310,Яблоня домашняя,2,1.5х2,0.03х2,Хорошее,Удаление
9,314,Алыча,1.7 м.п.,2.0,0.07,Хорошее,Удаление


In [12]:
# из документации pyautocad

from pyautocad import Autocad, APoint

acad = Autocad()
p1 = APoint(0, 0)
for i in range(5):
    obj = acad.model.AddText(u'Hi %s!' % i, p1, 2.5)
    p1.y += 10

# вывод: делаю модуль для рисования таблицы с помощью линий и текста